In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from dataset import extract_users_movies_ratings_lists, TripletDataset, extract_matrix_users_movies_ratings
import torch 

#Useful constants
number_of_users, number_of_movies = (10000, 1000)
RANDOM_STATE = 58
DATA_DIR = '../data'


data_pd = pd.read_csv(DATA_DIR+'/data_train.csv')
train_pd, val_pd = train_test_split(data_pd, train_size=0.9, random_state=RANDOM_STATE)


#users_train, movies_train, ratings_train = extract_users_movies_ratings_lists(train_pd)
#users_val, movies_val, ratings_val = extract_users_movies_ratings_lists(val_pd)
matrix_train, _ = extract_matrix_users_movies_ratings(train_pd)
matrix_val, _ = extract_matrix_users_movies_ratings(val_pd)

/home/gio/.local/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
K=6
u = np.random.rand(number_of_users, K)
v = np.random.rand(number_of_movies, K)



In [7]:
def step(u, v, y):
    u_new = np.empty_like(u)
    v_new = np.empty_like(v)


    for i in range(0, number_of_users):
        uv = np.matmul(u[i],v.transpose())
        a = np.matmul(v.transpose(), y[i, :]/uv)
        
        b = v.sum(axis=0)
        u_new[i] = u[i]* a / b
            
        
    u = u_new
    for j in range(0, number_of_movies):
        uv = np.matmul(v[j],u.transpose())
        a = np.matmul(u.transpose(), y[:, j]/uv)
        
        b = u.sum(axis=0)
        v_new[j] = v[j]*a/b
     
            
    return u_new, v_new


In [21]:
matrix_train.shape

(10000, 1000)

In [25]:
np.mean(matrix_train, axis=1).shape

(10000,)

In [26]:
def step(u, v, y):
    u_new = np.empty_like(u)
    v_new = np.empty_like(v)

    uv = np.matmul(u,v.transpose())
    a = np.matmul(v.transpose(), (y/uv).transpose()) #K,num_users
    b = v.sum(axis=0) #K
    print('b', b.shape)
    
    c = a.transpose()/b
    u_new = u * c
            
        
    u = u_new

    uv = np.matmul(u,v.transpose())
    a = np.matmul(u.transpose(), (y/uv)) #K, num_movies
    b = u.sum(axis=0) #K

    c = a.transpose() / b
    v_new = v * c
     
            
    return u_new, v_new


In [5]:
def rmse(y, yhat):
    mask = y.astype(np.bool)
    return np.sqrt(np.mean((y-yhat)[mask]**2))

In [27]:
old_score = np.inf
n = 0
while True:
    u, v = step(u, v, matrix_train)
    pred = np.matmul(u,v.transpose())
    val_score = rmse(matrix_val, pred)
    train_score = rmse(matrix_train, pred)

   
    print(f'Train rmse: {train_score}, val rmse: {val_score}')
    break
    if n % 1000 == 0:
        if val_score < old_score:
            np.save('./best_u', u)
            np.save('./best_v', v)
            old_score = val_score
        np.save('./u', u)
        np.save('./v', v)
    n+=1


b (6,)
Train rmse: 3.0851889607586123, val rmse: 3.1200740684328876


/tmp/ipykernel_57618/4172897665.py:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  mask = y.astype(np.bool)
